# Studying a set of tracks

In [ ]:
import huracanpy
import numpy as np
import seaborn as sns

## Load tracks
Here we show an example with a csv file that is embedded within HuracanPy for example. HuracanPy supports many track files format, see [huracanpy.load guide](../load.ipynb).

In [ ]:
# Loading the ERA5 1996 TC tracks
# The tracks detected by TempestExtremes in ERA5 for the year 1996 are embedded within
# the package as an example.
file = huracanpy.example_year_file
print(file)

# Load the tracks with huracanpy.load.
# Here the file extension is '.csv', the function will automatically recognise how to
# open it.
tracks = huracanpy.load(file)

# The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding
# to each point.
# Variables indicate position in space and time, as well as additional attributes such
# as maximum wind speed and minimum slp.
tracks

## Adding info to the tracks
HuracanPy has several function to add useful information to the tracks (for full list, see [huracanpy.info](../../api/info.rst)). Here for example we add basin and SSHS category information.

In [ ]:
# Add basin
tracks = tracks.hrcn.add_basin()  # Add basin attribute
tracks.basin

In [ ]:
# Show distribution of TC points among basins (calling seaborn function, works better
# with categorical labels)
sns.countplot(tracks.basin)

In [ ]:
# Add SSHS and pressure categories
tracks = tracks.hrcn.add_saffir_simpson_category(wind_name="wind10", wind_units="m s-1")
tracks = tracks.hrcn.add_pressure_category(
    slp_name="slp",
)
## (In ERA5 data, wind is stored in wind10 in m/s)
tracks[["saffir_simpson_category", "pressure_category"]]

In [ ]:
# Show distribution of TC points among categories (using xarray's built-in function)
tracks.saffir_simpson_category.plot.hist(
    bins=[-1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5], alpha=0.5
)
tracks.pressure_category.plot.hist(
    bins=[-1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5], alpha=0.5
)

## Plotting
HuracanPy embeds basic plotting functions, which are mainly meant for having a preliminary look at your data. In particular here we show how to plot the track points themselves, and track density. The [example gallery](../../examples/index.rst) displays nice plots made from HuracanPy and the associated scripts.
### Plotting the tracks

In [ ]:
# Plot ERA5 tracks colored by wind intensity
tracks.hrcn.plot_tracks(
    intensity_var_name="wind10",
)

### Plotting track density

In [ ]:
# You can plot the track density directly with `plot_density`, which is based on a
# simple 2D histogram of TC points
tracks.hrcn.plot_density()

In [ ]:
# You can also get the underlying density matrix with `get_density` and then use it to
# make you own plots in your favourite way
tracks.hrcn.get_density()

### Plotting genesis points

In [ ]:
# `get_gen_vals` allows you to subset only the genesis points in an efficient way
gen_points = tracks.hrcn.get_gen_vals()
gen_points

In [ ]:
# If you use `plot_tracks` on these, you can display only the genesis points.
gen_points.hrcn.plot_tracks()

## Compute statistics

### Number of cyclones

In [ ]:
tracks.track_id.hrcn.nunique()  # Count number of unique track ids

### Cyclone duration & TC days

In [ ]:
## Get the duration for each track
TC_duration = tracks.hrcn.get_track_duration()
TC_duration  # xarray.Dataset with track_id as dimension

In [ ]:
## Compute the total number of TC days
## Sum all the durations (and divide by 24 because durations are in hours)
TC_duration.sum() / 24

### Cyclone Intensity

In [ ]:
# There are two ways to obtain the lifetime maximum intensity (LMI) of each tracks
## 1. Use `get_apex_vals`, which return the subset of points only as specified LMI
tracks.hrcn.get_apex_vals(var_name="wind10")

In [ ]:
## 2. Compute lifetime maximum intensity per track with xarray's groupby
LMI_wind = tracks.wind10.groupby(tracks.track_id).max()
LMI_wind  # xarray.Dataset with track_id as dimension

In [ ]:
# You can then plot the LMI distribution using xarray's built-in plot function.
LMI_wind.plot.hist()

### ACE

In [ ]:
# Compute ACE for each point
tracks = tracks.hrcn.add_ace(wind_name="wind10", wind_units="m s**-1")
tracks.ace

In [ ]:
## Compute total ACE
tracks.ace.sum()

## Compositing lifecycle

In [ ]:
# Add time from apex variable to be able to superimpose all the tracks centered on apex
tracks = tracks.hrcn.add_time_from_apex(
    intensity_var_name="slp", stat="min"
)  # Add time from minimum pressure
tracks.time_from_apex

In [ ]:
tracks.time_from_apex / np.timedelta64(1, "h")

In [ ]:
# Plot composite SLP lifecycle
## Convert time_from_apex to hours
tracks["time_from_apex"] = tracks.time_from_apex / np.timedelta64(1, "h")
## Use xarray's where to mask points too far away from apex (48 hours away)
tracks_close_to_apex = tracks.where(np.abs(tracks.time_from_apex) <= 48, drop=True)
## Seaborn lineplot allows for drawing composites with uncertainty range
sns.lineplot(
    x=tracks_close_to_apex.time_from_apex,  # x-axis is time from apex
    y=tracks_close_to_apex.slp / 100,
)  # y-axis is slp, converted to hPa